# CareerFoundry Exercise 4.8 Grouping Data & Aggregating Variables

## 1. Importing libraries and data

In [1]:
#Importing libraries

import os
import numpy as np
import pandas as pd

In [2]:
path = r'C:\Users\rmr1q.000\Instacart Basket Analysis'
ords_prods_merged = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_new_columns.pkl'))

## 2. Finding the aggregated mean of order numbers for entire data set

In [3]:
ords_prods_merged.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


This time the result shows all of the departments. In the reading, we were only working with 1 million rows of data so not all of the departments were represented.

The values for the departments are also slightly different than in the reading because there are more orders from those departments which affects the result.

## 4. Creating a loyalty flag

In [4]:
#Creating a new column "max_orders" that shows how many orders each user has placed

ords_prods_merged['max_order'] = ords_prods_merged.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\rmr1q.000\AppData\Local\Temp\ipykernel_11084\4039568899.py:3: FutureWarning: The provided callable <function amax at 0x000001F2B07C3BA0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merged['max_order'] = ords_prods_merged.groupby(['user_id'])['order_number'].transform(np.max)


In [5]:
#Confirming the new column has been added

ords_prods_merged.head()

,Unnamed: 0,Index,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,...,_merge,price_range_loc,busiest day,busiest_period_of_day,max_order,loyalty_flag,spending_average,spending_flag,frequency_median,frequency_flag
0,0,0,2539329,1,1,2,8,NaN,196,1,...,both,Mid-range product,Regular busy,Average orders,10,New customer,6.367797,Low spender,20.5,Frequent Customer
1,1,1,2398795,1,2,3,7,15.0,196,1,...,both,Mid-range product,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Frequent Customer
2,2,2,473747,1,3,3,12,21.0,196,1,...,both,Mid-range product,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Frequent Customer
3,3,3,2254736,1,4,4,7,29.0,196,1,...,both,Mid-range product,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Frequent Customer
4,4,4,431534,1,5,4,15,28.0,196,1,...,both,Mid-range product,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Frequent Customer


In [6]:
#Using the .loc() function to create the loyalty flags

In [7]:
ords_prods_merged.loc[ords_prods_merged['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [8]:
ords_prods_merged.loc[(ords_prods_merged['max_order'] > 10) & (ords_prods_merged['max_order'] <= 40), 'loyalty_flag'] = 'Regular Customer'

In [9]:
ords_prods_merged.loc[ords_prods_merged['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [10]:
#Chceking the frequency distribution

ords_prods_merged['loyalty_flag'].value_counts()

loyalty_flag
Regular Customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

## 5. Identifying basic statistics between the different loyalty groups|

In [11]:
ords_prods_merged.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']})

prices              
                       mean  min      max
loyalty_flag                             
Loyal customer    10.386336  1.0  99999.0
New customer      13.294670  1.0  99999.0
Regular Customer  12.495717  1.0  99999.0

## Investigating the max value

In [12]:
#It seems interesting that there is a product that costs 999999 so I wanted to investigate which product(s) have this price

over_priced = ords_prods_merged.loc[ords_prods_merged['prices'] == 99999]

In [13]:
over_priced['product_name'].value_counts()

product_name
2 % Reduced Fat  Milk    698
Name: count, dtype: int64

It seems that 2% Reduced Fat Milk is very expensive at this store :O

## 6. Identifying high and low spenders

In [14]:
#Creating a new column for the spending averages for each customer

ords_prods_merged['spending_average'] = ords_prods_merged.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\rmr1q.000\AppData\Local\Temp\ipykernel_11084\1946724490.py:3: FutureWarning: The provided callable <function mean at 0x000001F2B07CC900> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  ords_prods_merged['spending_average'] = ords_prods_merged.groupby(['user_id'])['prices'].transform(np.mean)


In [15]:
#Creating a new flag for high spenders and low spenders based on the spending average

In [16]:
ords_prods_merged.loc[ords_prods_merged['spending_average'] < 10, 'spending_flag'] = 'Low spender'

In [17]:
ords_prods_merged.loc[ords_prods_merged['spending_average'] >= 10, 'spending_flag'] = 'High spender'

In [18]:
#Checking the frequency distribution

ords_prods_merged['spending_flag'].value_counts()

spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

## 7. Identifying frequent and non-frequent customers

In [27]:
#Creating a new column for the median number of days since a customer's last order

ords_prods_merged['frequency_median'] = ords_prods_merged.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\rmr1q.000\AppData\Local\Temp\ipykernel_11084\1422390895.py:3: FutureWarning: The provided callable <function median at 0x000001F2B0906480> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  ords_prods_merged['frequency_median'] = ords_prods_merged.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [28]:
#Checking the data frame for the new column

ords_prods_merged.head()

,Unnamed: 0,Index,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,...,_merge,price_range_loc,busiest day,busiest_period_of_day,max_order,loyalty_flag,spending_average,spending_flag,frequency_median,frequency_flag
0,0,0,2539329,1,1,2,8,NaN,196,1,...,both,Mid-range product,Regular busy,Average orders,10,New customer,6.367797,Low spender,20.5,Frequent Customer
1,1,1,2398795,1,2,3,7,15.0,196,1,...,both,Mid-range product,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Frequent Customer
2,2,2,473747,1,3,3,12,21.0,196,1,...,both,Mid-range product,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Frequent Customer
3,3,3,2254736,1,4,4,7,29.0,196,1,...,both,Mid-range product,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Frequent Customer
4,4,4,431534,1,5,4,15,28.0,196,1,...,both,Mid-range product,Least busy,Average orders,10,New customer,6.367797,Low spender,20.5,Frequent Customer


In [29]:
#Creating new flag for frequency of customers based on the median frequency since their last order

In [30]:
ords_prods_merged.loc[ords_prods_merged['frequency_median'] > 20, 'frequency_flag'] = 'Non-frequent Customer'

In [31]:
ords_prods_merged.loc[(ords_prods_merged['frequency_median'] <= 20) & (ords_prods_merged['frequency_median'] > 10), 'frequency_flag'] = 'Regular Customer'

In [32]:
ords_prods_merged.loc[ords_prods_merged['frequency_median'] <= 10, 'frequency_flag'] = 'Frequent Customer'

In [33]:
#Checking the frequency distribution

ords_prods_merged['frequency_flag'].value_counts()

frequency_flag
Frequent Customer        21559853
Regular Customer          7208564
Non-frequent Customer     3636437
Name: count, dtype: int64

## 9. Exporting the data

In [34]:
#Overwriting the previous data file because there are more new columns and having too many files can be problematic

ords_prods_merged.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_merged_new_columns.pkl'))